In [1]:
# Create a model for text classification using a neural network to assign class probabilities
import pandas as pd
import numpy as np
import re
# from tensorflow.contrib import learn
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
# from gensim.parsing.preprocessing import remove_stopwords
# import Pattern


##### RUN FOLLOWING TO GENERATE THE CSV FILE WITH TWEET CONTENT AND HASHTAGS
##### OTHERWISE LOAD THE SAVED DATAFRAME

# data = pd.read_csv('../data/ira_tweets_new/rus_troll_tweet_text.csv')
#english_tweets = data[data.tweet_language == 'en']
#hashtag_info = pd.read_csv('../data/ira_tweets_new/rus_troll_tweet_stats.csv')[['tweetid', 'hashtags']]
#merged = pd.merge(english_tweets, hashtag_info, on='tweetid')[['tweet_text', 'hashtags']]

# # Remove tweets that have no hashtag
# data.replace('[]',np.nan, inplace=True)
# data.dropna(how='any', inplace=True)

# # Remove hashtags and other tags and grammar stuff
# data_clean=data.replace('#\w+', '', regex=True).replace('@\w+', '', regex=True).replace('RT ', '', regex=True)
# data_clean.tweet_text = data_clean.tweet_text.apply(lambda x: re.sub(r'http\S+', '', x)).apply(lambda x: re.sub(r"'|\"|`|:|\?|~|,|\.", '', x))
# data_clean.hashtags = data_clean.hashtags.apply(lambda x: x[1:-1]) # make list in hashtag column
# duplicated = pd.DataFrame(data_clean.hashtags.str.split(', ').tolist(), index=data_clean.tweet_text).stack()
# duplicated = duplicated.reset_index()[[0, 'tweet_text']] # var1 variable is currently labeled 0
# duplicated.columns = ['hashtags', 'tweet_text'] # renaming var1

# # Show top hashtags
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     print(duplicated.hashtags.value_counts()[0:300])
    
# # Remove stop words
# duplicated.tweet_text = duplicated.tweet_text.apply(lambda x: remove_stopwords(x))


# # Filter dataframe for hashtags identified to be in a topic

# # Create list of relevant hashtags
# hashtag_list = ['MAGA', 'WakeUpAmerica', 'MakeAmericaGreatAgain', 
#                 'ARRESTObama', 'TrumpTrain', 'pjnet', 'PJNET', 'Trump2016',
#                 'maga', 'TrumpForPresident', 'TrumpBecause', 'ObamaLameDuck',
#                 'ICelebrateTrumpWith', 'BlackLivesMatter', 'BlackTwitter', 'blacklivesmatter', 
#                 'BlackHistoryMonth', 'BlackSkinIsNotACrime',  'blacktolive', 'BlackToLive', 'blacktwitter',
#                 'tech', 'science', 'entertainment', 'celebs', 'MyEmmyNominationWouldBe', 'soundcloud', 'iTunes',
#                 'rap', 'health', 'sports', 'hockey', 'Sports', 'MyOlympicSportWouldBe', 'ISIS', 'IslamKills',
#                 'StopIslam', 'Hillary', 'ThingsMoreTrustedThanHillary', 'NeverHillary', 'CrookedHillary',
#                 'world', 'Syria', 'Turkey', 'Brussels', 'Russia', 'Iraq', 'business', 'news', 'local', 
#                 'Fukushima2015', 'FukushimaAgain', 'FUKUSHIMA2015', 'fukushima2015', 'IHatePokemonGoBecause',
#                 'crime', 'PoliceBrutality', 'USFA', 'ArmyWomenProblems']

# # Filter dataframe
# filtered = duplicated[duplicated.hashtags.isin(hashtag_list)]


# # Lemmatize the words and also keeping their context (verb, noun etc)

# from gensim.utils import lemmatize
# filtered.tweet_text = filtered.tweet_text.apply(lambda x: lemmatize(x, min_length=3))


# # change hashtag names to the topic names
# pd.options.mode.chained_assignment = None  # default='warn' 

# # General News
# filtered.hashtags.loc[filtered[(filtered.hashtags == 'news') |
#                                (filtered.hashtags == 'local')].index.values] = 'General News'

# # Fukushima
# filtered.hashtags.loc[filtered[(filtered.hashtags == 'Fukushima2015') |
#                                (filtered.hashtags == 'FukushimaAgain') |
#                               (filtered.hashtags == 'FUKUSHIMA2015') | 
#                               (filtered.hashtags == 'fukushima2015')].index.values] = 'Fukushima'

# # Crime
# filtered.hashtags.loc[filtered[(filtered.hashtags == 'crime') |
#                                (filtered.hashtags == 'PoliceBrutality')].index.values] = 'Crime'

# # Gun Related
# filtered.hashtags.loc[filtered[(filtered.hashtags == 'USFA') |
#                                (filtered.hashtags == 'ArmyWomenProblems')].index.values] = 'Gun Related'

# # Anti-Islam
# filtered.hashtags.loc[filtered[(filtered.hashtags == 'ISIS') |
#                                (filtered.hashtags == 'IslamKills') |
#                               (filtered.hashtags == 'StopIslam')].index.values] = 'Anti-Islam'

# # Anti-Hillary
# filtered.hashtags.loc[filtered[(filtered.hashtags == 'Hillary') |
#                                (filtered.hashtags == 'ThingsMoreTrustedThanHillary') |
#                               (filtered.hashtags == 'NeverHillary') |
#                               (filtered.hashtags == 'CrookedHillary')].index.values] = 'Anti-Hillary'


# # Foreign Countries
# filtered.hashtags.loc[filtered[(filtered.hashtags == 'world') |
#                                (filtered.hashtags == 'Syria') |
#                               (filtered.hashtags == 'Turkey') |
#                               (filtered.hashtags == 'Brussels') |
#                               (filtered.hashtags == 'Russia') |
#                               (filtered.hashtags == 'Iraq')].index.values] = 'Foreign Countries'

# # Entertainment
# filtered.hashtags.loc[filtered[(filtered.hashtags == 'entertainment') |
#                                (filtered.hashtags == 'celebs') |
#                               (filtered.hashtags == 'MyEmmyNominationWouldBe') |
#                               (filtered.hashtags == 'soundcloud') |
#                               (filtered.hashtags == 'iTunes') |
#                               (filtered.hashtags == 'rap')].index.values] = 'Entertainment'


# # Black Race Support
# filtered.hashtags.loc[filtered[(filtered.hashtags == 'BlackLivesMatter') |
#                                (filtered.hashtags == 'BlackTwitter') |
#                               (filtered.hashtags == 'blacklivesmatter') |
#                               (filtered.hashtags == 'BlackHistoryMonth') |
#                               (filtered.hashtags == 'BlackSkinIsNotACrime') |
#                               (filtered.hashtags == 'blacktolive') |
#                               (filtered.hashtags == 'BlackToLive') |
#                               (filtered.hashtags == 'blacktwitter')].index.values] = 'Black Support'

# # Trump Support
# filtered.hashtags.loc[filtered[(filtered.hashtags == 'MAGA') |
#                                (filtered.hashtags == 'WakeUpAmerica') |
#                               (filtered.hashtags == 'MakeAmericaGreatAgain') |
#                               (filtered.hashtags == 'ARRESTObama') |
#                               (filtered.hashtags == 'TrumpTrain') |
#                               (filtered.hashtags == 'pjnet') |
#                               (filtered.hashtags == 'PJNET') |
#                               (filtered.hashtags == 'Trump2016') |
#                               (filtered.hashtags == 'maga') |
#                               (filtered.hashtags == 'TrumpForPresident') |
#                               (filtered.hashtags == 'TrumpBecause') |
#                               (filtered.hashtags == 'ObamaLameDuck') |
#                               (filtered.hashtags == 'ICelebrateTrumpWith')].index.values] = 'Trump Support'


# # Technology
# filtered.hashtags.loc[filtered[(filtered.hashtags == 'tech') |
#                                (filtered.hashtags == 'science')].index.values] = 'Technology'

# # Health
# filtered.hashtags.loc[filtered[filtered.hashtags == 'health'].index.values] = 'Health'

# # Sports
# filtered.hashtags.loc[filtered[(filtered.hashtags == 'sports') |
#                                (filtered.hashtags == 'hockey') |
#                               (filtered.hashtags == 'Sports') |
#                               (filtered.hashtags == 'MyOlympicSportWouldBe')].index.values] = 'Sports'


# # Business
# filtered.hashtags.loc[filtered[filtered.hashtags == 'business'].index.values] = 'Business'

# # Video GAmes
# filtered.hashtags.loc[filtered[filtered.hashtags == 'IHatePokemonGoBecause'].index.values] = 'Video Games'

In [77]:
# Load english dataframe
filtered = pd.read_csv('../data/tweets_n_hashtags.csv', index_col = 0)

In [78]:
filtered.head()

,hashtags,tweet_text
33,General News,"[b'john/NN', b'carroll/NN', b'university/NN', ..."
34,General News,"[b'free/JJ', b'spring/NN', b'cook/VB', b'book/..."
37,General News,"[b'rocky/JJ', b'river/NN', b'prepare/VB', b'fl..."
39,General News,"[b'forecast/VB', b'cooler/JJ', b'mid/JJ', b'we..."
42,General News,"[b'arrest/VB', b'connection/NN', b'art/NN', b'..."


In [79]:
filtered.tweet_text = filtered.tweet_text.apply(lambda x: x[1:-1]).replace("b'", '', regex=True).replace("'", '', regex=True)
filtered.tweet_text = filtered.tweet_text.apply(lambda x: x.split(', '))

In [80]:
filtered

,hashtags,tweet_text
33,General News,"[john/NN, carroll/NN, university/NN, get/VB, g..."
34,General News,"[free/JJ, spring/NN, cook/VB, book/NN, healthy..."
37,General News,"[rocky/JJ, river/NN, prepare/VB, flooding/JJ, ..."
39,General News,"[forecast/VB, cooler/JJ, mid/JJ, week/NN, cool..."
42,General News,"[arrest/VB, connection/NN, art/NN, festival/NN..."
44,General News,"[horror/NN, director/NN, craven/JJ, die/VB, sc..."
45,Sports,"[secretariat/VB, dominate/VB, american/JJ, pha..."
46,General News,"[black/JJ, box/NN, miss/VB, german/NN, airline..."
48,Sports,"[dale/NN, earnhardt/NN, get/VB, engage/VB, gir..."
50,Sports,"[brown/NN, mingo/NN, undergo/VB, surgery/NN, b..."


In [28]:
categories = ['General News', 'Sports', 'Crime', 'Technology', 'Entertainment', 
              'Anti-Hillary', 'Fukushima', 'Foreign Countries', 'Trump Support',
              'Gun Related', 'Video Games', 'Health', 'Business',
              'Black Support', 'Anti-Islam']
from collections import Counter

vocab = Counter()

for text in filtered.tweet_text:
    for word in text: ## change this because the texts are already a list of words
        vocab[word[2:-1].lower()]+=1

total_words = len(vocab)

def get_word_2_index(vocab):
    word2index = {}
    for i,word in enumerate(vocab):
        word2index[word.lower()] = i

    return word2index

word2index = get_word_2_index(vocab)

In [32]:
total_words

87510

In [63]:
def get_batch(df,i,batch_size):
    batches = []
    results = []
    texts = df.tweet_text[i*batch_size:i*batch_size+batch_size]
    categories = df.hashtags[i*batch_size:i*batch_size+batch_size]
    for text in texts:
        layer = np.zeros(total_words,dtype=float) # total words is not defined
        for word in text:
            layer[word2index[word[2:-1].lower()]] += 1

        batches.append(layer)

    for category in categories:
        index_y = -1
        if category == 'General News':
            index_y = 0
        elif category == 'Sports':
            index_y = 1
        elif category == 'Crime':
            index_y = 2
        elif category == 'Technology':
            index_y = 3
        elif category == 'Entertainment':
            index_y = 4
        elif category == 'Anti-Hillary':
            index_y = 5
        elif category == 'Fukushima':
            index_y = 6
        elif category == 'Foreign Countries':
            index_y = 7
        elif category == 'Trump Support':
            index_y = 8
        elif category == 'Gun Related':
            index_y = 9
        elif category == 'Video Games':
            index_y = 10
        elif category == 'Health':
            index_y = 11
        elif category == 'Business':
            index_y = 12
        elif category == 'Black Support':
            index_y = 13
        elif category == 'Anti-Islam':
            index_y = 14
        results.append(index_y)


    return np.array(batches),np.array(results)

In [44]:
# Parameters
learning_rate = 0.01
num_epochs = 4
batch_size = 150
display_step = 1

# Network Parameters
hidden_size = 100      # 1st layer and 2nd layer number of features
input_size = total_words # Words in vocab
num_classes = 15        # Categories: graphics, sci.space and baseball

In [45]:
loss = nn.CrossEntropyLoss()
inputs = Variable(torch.randn(2, 5), requires_grad=True)
target = Variable(torch.LongTensor(2).random_(5))
output = loss(inputs, target)
output.backward()

# Do Neural Network with Linear Layers

In [46]:
class OurNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.layer_1 = nn.Linear(input_size,hidden_size, bias=True)
        self.relu = nn.ReLU()
        self.layer_2 = nn.Linear(hidden_size, hidden_size, bias=True)
        self.output_layer = nn.Linear(hidden_size, num_classes, bias=True)

    def forward(self, x):
        out = self.layer_1(x)
        out = self.relu(out)
        out = self.layer_2(out)
        out = self.relu(out)
        out = self.output_layer(out)
        return out
    
net = OurNet(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, momentum=0.9)
# scheduler = ReduceLROnPlateau(optimizer, 'min') # for validation set

In [47]:
## Train the Model
for epoch in range(num_epochs):
    total_batch = int(len(filtered.tweet_text)/batch_size)
    # Loop over all batches
    for i in range(total_batch):
        batch_x,batch_y = get_batch(filtered,i,batch_size)
        articles = Variable(torch.FloatTensor(batch_x))
        labels = Variable(torch.LongTensor(batch_y))
        #print("articles",articles)
        #print(batch_x, labels)
        #print("size labels",labels.size())

        # Forward + Backward + Optimize
        optimizer.zero_grad()  # zero the gradient buffer
        outputs = net(articles)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                   %(epoch+1, num_epochs, i+1, len(filtered.tweet_text)//batch_size, loss.data[0]))
            
    

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch [1/4], Step [100/4413], Loss: 1.0593
Epoch [1/4], Step [200/4413], Loss: 1.1449
Epoch [1/4], Step [300/4413], Loss: 0.8779
Epoch [1/4], Step [400/4413], Loss: 0.8435
Epoch [1/4], Step [500/4413], Loss: 0.6738
Epoch [1/4], Step [600/4413], Loss: 0.7350
Epoch [1/4], Step [700/4413], Loss: 0.9442
Epoch [1/4], Step [800/4413], Loss: 0.8077
Epoch [1/4], Step [900/4413], Loss: 0.8551
Epoch [1/4], Step [1000/4413], Loss: 1.1576
Epoch [1/4], Step [1100/4413], Loss: 0.8019
Epoch [1/4], Step [1200/4413], Loss: 0.7339
Epoch [1/4], Step [1300/4413], Loss: 0.9920
Epoch [1/4], Step [1400/4413], Loss: 0.6972
Epoch [1/4], Step [1500/4413], Loss: 0.7185
Epoch [1/4], Step [1600/4413], Loss: 0.7443
Epoch [1/4], Step [1700/4413], Loss: 0.7170
Epoch [1/4], Step [1800/4413], Loss: 0.9561
Epoch [1/4], Step [1900/4413], Loss: 0.7751
Epoch [1/4], Step [2000/4413], Loss: 0.7682
Epoch [1/4], Step [2100/4413], Loss: 0.8138
Epoch [1/4], Step [2200/4413], Loss: 0.8708
Epoch [1/4], Step [2300/4413], Loss: 0.77

In [50]:
# lemmatize(['I need to get healthy who wants to go to the gym', 'Yesterday I saw a great football game'], min_length=3)

NameError: name 'lemmatize' is not defined

In [68]:
test_set = filtered.loc[2150651].copy()

test_set

test_set.head()
batch_x_test,batch_y_test = get_batch(test_set,0,2)
articles = Variable(torch.FloatTensor(batch_x_test))
labels = torch.LongTensor(batch_y_test)
outputs = net(articles)
print(outputs)
_, predicted = torch.max(outputs.data, 1)
print(predicted)

# torch.save(net.state_dict(), 'supervised_linear')






















tensor([[ -2.1686,  -0.3003, -11.8906, -10.8411,   1.0247,  -5.6979, -20.9244,
           2.4551,  18.9812, -24.1216, -24.0897, -25.0826,  -7.8877,  -3.5073,
          -6.3526],
        [ -3.6174,  -0.8996, -14.0025, -14.0994,   0.6767,  -7.0539, -26.6901,
           2.9211,  26.0714, -31.7085, -31.5246, -32.6922, -10.4274,  -3.9084,
          -8.3163]], grad_fn=<ThAddmmBackward>)
tensor([8, 8])


# Try Convolutional Neural Net

In [ ]:
class ConvNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.conv1 = nn.Conv1d(input_size, input_size, kernel_size=2) # dimensions need to be fixed
        self.conv2 = nn.Conv1d(hidden_size, hidden_size, kernel_size=2)
        self.mp = nn.MaxPool1d(2)
        self.fc = nn.Linear(100, 15)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.mp(x)
        x = F.relu(x)
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)
        x = self.fc(x)
        return F.log_softmax(x)
        
convnet = ConvNet(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [ ]:
## Train the Model
for epoch in range(num_epochs):
    total_batch = int(len(filtered.tweet_text)/batch_size)
    # Loop over all batches
    for i in range(total_batch):
        batch_x,batch_y = get_batch(filtered,i,batch_size)
        articles = Variable(torch.FloatTensor(batch_x))
        labels = Variable(torch.LongTensor(batch_y))
        #print("articles",articles)
        #print(batch_x, labels)
        #print("size labels",labels.size())

        # Forward + Backward + Optimize
        optimizer.zero_grad()  # zero the gradient buffer
        outputs = convnet(articles)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                   %(epoch+1, num_epochs, i+1, len(filtered.tweet_text)//batch_size, loss.data[0]))